# ACIT5900: Master Thesis
### *Data Preprocessing and Cleaning*

>-------------------------------------------
> *Spring 2025*

>--------------------------------------------

<a id="top"></a>
1. [**Data Preprocessing**](#processing)<br>
    1.1 [**Read and Inspect Data**](#read) <br>
    1.2 [**Handle Missing Values**](#missing) <br>
    1.3 [**Extract Abstracts**](#abstract) <br>
2. [**Data Cleaning**](#cleaning)<br>
    2.1 [**Cleaning of Text Content**](#text) <br>
    2.2 [**Cleaning of Author Column**](#author) <br>
3. [**Save DataFrame as CSV**](#csv)<br>




In [1]:
!pip install networkx nameparser


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# import modules needed
import re
import pandas as pd

## <a id="processing"></a> 1) Data Preprocessing

### <a id="read"></a> 1.1) Read and Inspect Data

This section loads the combined dataset from a CSV file and performs a quick inspection to understand its structure. The `head()` function displays the first few rows to provide a snapshot of the data, while `dtypes` reveals the data types of each column, helping identify any necessary preprocessing steps. Finally, the number of duplicated rows is counted to check for potential redundancy in the dataset, which may need to be addressed before further processing.

[⬆️ Back to Top](#top)

In [3]:
# read data
df = pd.read_csv('df_combined.csv')
df.head()

,title,authors,year_published,number,volume,journal,type,content,doi,file
0,Contrastive autoencoder for anomaly detection ...,Hao Zhou and Ke Yu and Xuan Zhang and Guanlin ...,2022,NaN,610,Information Sciences,article,Contrastive autoencoder for anomaly detection ...,https://doi.org/10.1016/j.ins.2022.07.179,doc16
1,Detection of abnormality in wireless capsule e...,Samir Jain and Ayan Seal and Aparajita Ojha an...,2020,NaN,127,Computers in Biology and Medicine,article,Computers in Biology and Medicine 127 (2020) 1...,https://doi.org/10.1016/j.compbiomed.2020.104094,doc17
2,Single image dehazing using a new color channel,Geet Sahu and Ayan Seal and Ondrej Krejcar and...,2021,NaN,74,Journal of Visual Communication and Image Repr...,article,J. Vis. Commun. Image R. 74 (2021) 103008\nAva...,https://doi.org/10.1016/j.jvcir.2020.103008,doc15
3,Estimating Tukey depth using incremental quant...,Hugo L. Hammer and Anis Yazidi and Håvard Rue,2022,NaN,122,Pattern Recognition,article,Pattern Recognition 122 (2022) 108339 \nConten...,https://doi.org/10.1016/j.patcog.2021.108339,doc29
4,Advanced Passive Operating System Fingerprinti...,"Hagos, Desta Haileselassie and Løland, Martin ...",2020,NaN,NaN,NaN,inproceedings,Advanced Passive Operating System Fingerprinti...,10.1109/ICCCN49398.2020.9209694,doc28


In [4]:
# check types
df.dtypes

title              object
authors            object
year_published      int64
number            float64
volume             object
journal            object
type               object
content            object
doi                object
file               object
dtype: object

In [5]:
# check for duplicates
df.duplicated().sum()

np.int64(0)

### <a id="missing"></a> 1.2) Handle Missing Values

This section addresses missing data within the dataset. It begins by identifying which columns contain null values and how many entries are affected. To ensure consistency and avoid errors in downstream tasks, all missing values are replaced with the placeholder string `"missing"`. This simple imputation strategy preserves the dataset’s structure while allowing models to process records that would otherwise contain empty fields.

[⬆️ Back to Top](#top)

In [6]:
# check for missing values
df.isna().sum()

title              0
authors            0
year_published     0
number            25
volume             7
journal            6
type               0
content            0
doi                4
file               0
dtype: int64

In [7]:
# replace missing values
df = df.fillna('missing')
df.head()

,title,authors,year_published,number,volume,journal,type,content,doi,file
0,Contrastive autoencoder for anomaly detection ...,Hao Zhou and Ke Yu and Xuan Zhang and Guanlin ...,2022,missing,610,Information Sciences,article,Contrastive autoencoder for anomaly detection ...,https://doi.org/10.1016/j.ins.2022.07.179,doc16
1,Detection of abnormality in wireless capsule e...,Samir Jain and Ayan Seal and Aparajita Ojha an...,2020,missing,127,Computers in Biology and Medicine,article,Computers in Biology and Medicine 127 (2020) 1...,https://doi.org/10.1016/j.compbiomed.2020.104094,doc17
2,Single image dehazing using a new color channel,Geet Sahu and Ayan Seal and Ondrej Krejcar and...,2021,missing,74,Journal of Visual Communication and Image Repr...,article,J. Vis. Commun. Image R. 74 (2021) 103008\nAva...,https://doi.org/10.1016/j.jvcir.2020.103008,doc15
3,Estimating Tukey depth using incremental quant...,Hugo L. Hammer and Anis Yazidi and Håvard Rue,2022,missing,122,Pattern Recognition,article,Pattern Recognition 122 (2022) 108339 \nConten...,https://doi.org/10.1016/j.patcog.2021.108339,doc29
4,Advanced Passive Operating System Fingerprinti...,"Hagos, Desta Haileselassie and Løland, Martin ...",2020,missing,missing,missing,inproceedings,Advanced Passive Operating System Fingerprinti...,10.1109/ICCCN49398.2020.9209694,doc28


### <a id="abstract"></a> 1.3) Extract Abstracts

This section focuses on extracting abstracts from research articles within the dataset. It first filters out documents that do not contain the keyword "abstract" (case-insensitive) to ensure only relevant texts are processed. A regular expression is then used to extract the abstract section by identifying text between the word "abstract" and common structural markers like "introduction", "keywords", or section numbers. The extracted abstract is stored in a separate column, while the original content is updated to remove the abstract portion, avoiding redundancy and preparing the text for cleaner downstream processing.

[⬆️ Back to Top](#top)

In [8]:
# check which articles that contain abstract
df['content'].str.contains(r'(?i)\babstract\b', regex=True)

0     False
1     False
2     False
3     False
4      True
5      True
6      True
7      True
8     False
9      True
10     True
11    False
12    False
13    False
14    False
15     True
16     True
17    False
18    False
19    False
20    False
21    False
22     True
23     True
24    False
25    False
26     True
27    False
28     True
29    False
30     True
31     True
32     True
33     True
34    False
35     True
36    False
Name: content, dtype: bool

In [9]:
def extract_abstract(text):
    """
    Extract abstract from articles.
    """
    match = re.search(r'(?i)\babstract\b(.*?)(?=\b(introduction|keywords|1\.|I\.)\b|$)', text, re.DOTALL)
    return match.group(1).strip() if match else None

In [10]:
# keep only documents that have an abstract
df = df[df['content'].str.contains(r'(?i)\babstract\b', regex=True)].copy()

# extract the abstract
df['abstract'] = df['content'].apply(extract_abstract)

# remove the abstract from the content column
df['content'] = df.apply(lambda row: row['content'].replace(row['abstract'], '', 1) if row['abstract'] else row['content'], axis=1)
df

,title,authors,year_published,number,volume,journal,type,content,doi,file,abstract
4,Advanced Passive Operating System Fingerprinti...,"Hagos, Desta Haileselassie and Løland, Martin ...",2020,missing,missing,missing,inproceedings,Advanced Passive Operating System Fingerprinti...,10.1109/ICCCN49398.2020.9209694,doc28,"—Securing and managing large, complex enterpri..."
5,Artificial intelligence in the fertility clini...,"Riegler, M A and Stensen, M H and Witczak, O a...",2021,9.0,36,Human Reproduction,article,.................................................,10.1093/humrep/deab168,doc14,": In recent years, the amount of data produced..."
6,EvoDynamic: A Framework for the Evolution of G...,"Pontes-Filho, Sidney\nand Lind, Pedro\nand Yaz...",2020,missing,missing,missing,inproceedings,EvoDynamic: a framework for the evolution of\n...,missing,doc10,. Dynamical systems possess a computational ca...
7,A general representation of dynamical systems ...,Sidney Pontes{-}Filho and\nAnis Yazidi and\nJi...,2019,missing,abs/1907.01856,CoRR,article,A general representation of dynamical systems ...,missing,doc11,—Dynamical systems are capable of performing c...
9,Exploring Multilingual Word Embedding Alignmen...,"Aaby, Pernille\nand Biermann, Daniel\nand Yazi...",2023,missing,missing,missing,inproceedings,"\n \n \n \nAccepted manuscript \nAaby, P., Bi...",missing,doc12,". Contextual language models, such as transfor..."
10,A Deep Learning-Based Tool for Automatic Brain...,"Pontes-Filho, Sidney\nand Dahl, Annelene Gulde...",2022,missing,missing,missing,inproceedings,A deep learning based tool for automatic brain...,missing,doc9,—Removing skull artifacts from functional magn...
15,Genetic Algorithms For Tightening Security,"Palumbo, Fabrizio and Buji, Adam and Yazidi, A...",2022,missing,missing,missing,inproceedings,Genetic Algorithms For Tightening Security\n1s...,10.23919/WMNC56391.2022.9954297,doc7,—Proper conﬁguration of operating systems and ...
16,How (not to) Run an AI Project in Investigativ...,"M. Fridman, R. Krøvel and F. Palumbo",2023,0.0,0,Journalism Practice,article,Journalism Practice\nISSN: (Print) (Online) Jo...,10.1080/17512786.2023.2253797,doc3,Data journalists are increasingly reliant on a...
22,A New Adaptive Mixture Distance-Based Improved...,"Sharma, Krishna Kumar and Seal, Ayan and Yazid...",2022,missing,71,IEEE Transactions on Instrumentation and Measu...,article,IEEE TRANSACTIONS ON INSTRUMENTATION AND MEASU...,10.1109/TIM.2022.3216366,doc22,—With the rapid development of sensors and mec...
23,A sensitivity analysis of cellular automata an...,"Tom Eivind Glover, Ruben Jahren, Francesco Mar...",2025,1.0,40,"International Journal of Parallel, Emergent an...",article,"International Journal of Parallel, Emergent an...",10.1080/17445760.2024.2396334,doc34,Elementary Cellular Automata (ECA) are well-st...


## <a id="cleaning"></a> 2) Data Cleaning

### <a id="text"></a> 2.1) Cleaning of Text Content

This section applies text cleaning operations to prepare the article content and abstracts for processing. The `clean_text` function removes various non-informative or distracting elements commonly found in research documents, such as journal URLs, ISSN numbers, copyright and license notices, download banners, and inline citations. It also strips statistical annotations and normalizes whitespace to ensure consistent formatting. These cleaning steps help reduce noise in the dataset, making it easier for downstream models to focus on the actual research content. The cleaned versions are stored in separate columns to preserve the original text.

[⬆️ Back to Top](#top)

In [11]:
def clean_text(text):
    """
    Cleans raw document text by removing URLs, citations, metadata, and boilerplate info.

    Args:
        text: Raw text to clean.

    Returns:
        str: Cleaned text.
    """
    # remove journal homepage URLs
    text = re.sub(r'https?://\S+', '', text) 

    # remove ISSN numbers
    text = re.sub(r'ISSN:\s*\(.*?\)', '', text)

    # remove copyright/license info
    text = re.sub(r'©\s*\d{4}.*?license.*', '', text, flags=re.IGNORECASE)

    # remove unwanted "Downloaded from" lines or publication data
    text = re.sub(r'Downloaded from .+? on \w+ \d{2}, \d{4}', '', text)

    # remove any reference to URLs or links in text that aren’t essential
    text = re.sub(r'\bwww\.\S+', '', text) 

    # remove unwanted citations and references (vurder å fjerne dette)
    text = re.sub(r'REFERENCES AND NOTES.*', '', text, flags=re.DOTALL)  # remove references and notes section
    text = re.sub(r'\(\d+\)', '', text)  # remove inline citations 
    text = re.sub(r'\*\*P.*?Wilcoxon.*?\n', '', text)  # remove statistical results

    # normalize spaces and remove unwanted newlines
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [12]:
# apply cleaning
df["cleaned_content"] = df["content"].apply(clean_text)
df["cleaned_abstract"] = df["abstract"].apply(clean_text)

In [13]:
pd.set_option('display.max_colwidth', None)
df[['cleaned_content']].head(1)

cleaned_content
4  Advanced Passive Operating System Fingerprinting Using Machine Learning and Deep Learning Desta Haileselassie Hagos∗, Martin Løland†, Anis Yazidi‡, Øivind Kure §, Paal E. Engelstad ¶ ∗§¶University of Oslo, Department of Technology Systems, Kjeller, Norway ∗†‡¶Oslo Metropolitan University, Department of Computer Science, Oslo, Norway Email: ∗destahh@iﬁ.uio.no, †martin.loeland@gmail.com, {‡anis.yazidi, ¶paal.engelstad}@oslomet.no, §oivind.kure@its.uio.no Abstract Keywords—Operating System, Fingerprinting, Machine Learning, Deep Learning, Passive Measurements I. INTRODUCTION AND MOTIVATION As modern network infrastructures grow in size, collecting detailed relevant knowledge about the dynamic characteristics and complexity of large heterogeneous networks is crucial for many purposes e.g., network vulnerability assessment and monitoring, spam detection, etc. Developing advanced network security and monitoring techniques are important for both the research and security practitioners. There has been a signiﬁcant research work in the context of network management and cybersecurity on developing network security tools to ﬁngerprint remote Operating Systems (OSes) [26, 27, 28, 41, 42]. OS ﬁngerprinting is the process of inferring the OS of a machine operating with TC/IP by a remote device connected on the Internet without having physical access to the device [20]. There are many different custom tools for ﬁngerprinting of the most commonly used OSes based on the characteristics of its underlying TCP/IP network stack [20] and this, to a large extent, is due to variability in how the TCP/IP stack is traditionally implemented across different OSes [25]. One common approach, for example, is by collecting the TCP/IP stack basic parameters [23], e.g., IP initial Time To Live (TTL) default values [5], HTTP packets using the User-agent ﬁeld [22], Internet Control Message Protocol (ICMP) requests [29], known open port patterns, TCP window size [18], TCP Maximum Segment Size (MSS) [31], IP Don’t Fragment (DF) ﬂag [30], a set of other speciﬁc TCP options to mention a few. However, in our work, we want to take this one step further by combining these basic features and other settings with the underlying TCP variant as a feature in our model due to the fact that different OSes are doing slightly different implementations of TCP. Some implementations of common TCP variants quickly overshoot the size of the Congestion Window (cwnd) because of differences in the variant implementations. Hence, we believe that knowing the implementation of the underlying OS may help us understand better their exact behavior. It can also help us explore how to classify an OS when different OSes are implementing the same TCP variant. Fingerprinting Techniques: We can determine what OS a remote computer on the Internet is running by either passively listening to trafﬁc captured from a network or by actively sending it packets. The most widely used complementary remote OS ﬁngerprinting proven approaches that employ a variety of TCP/IP stack scanning are broadly categorized into classes of active and passive methods. • Active Fingerprinting: This technique is based on actively transmitting one or more specially crafted network packets with different packet settings or ﬂags to a remote network device in order to analyze the corresponding potentially identifying replies [26, 41]. This method determines knowledge of the underlying OS according to the received responses from the target device by examining the network behavior of known TCP/IP stack [35]. However, since this approach injects additional trafﬁc to the network by generating active probes, it may itself trigger alarms and get blocked by ﬁrewall rules and Network address translators (NATs) [8]. • Passive Fingerprinting: This approach, on the other hand, inspects and analyzes packets traveling between end hosts without injecting any trafﬁc into the network [27, 28, 42]. This technique with little res

### <a id="author"></a> 2.2) Cleaning of Author Column

This section handles the cleaning and normalization of author names. The process starts by removing formatting artifacts such as newline characters and placeholder symbols. It then splits author strings based on conjunctions like “and” and punctuation, and reformats names to follow a standard "First Last" structure. A normalization step ensures that different variants of the same name (e.g., initials, reordered formats) are unified. Special handling is included for problematic entries like multiple versions of “Gustavo Mello.” The result is a cleaned list of author names that is easier to work with for analysis or citation generation.

[⬆️ Back to Top](#top)<>

In [14]:
df['authors'].unique()[:17]

array(['Hagos, Desta Haileselassie and Løland, Martin and Yazidi, Anis and Kure, Øivind and Engelstad, Paal E.',
       'Riegler, M A and Stensen, M H and Witczak, O and Andersen, J M and Hicks, S A and Hammer, H L and Delbarre, E and Halvorsen, P and Yazidi, A and Holst, N and Haugen, T B',
       'Pontes-Filho, Sidney\nand Lind, Pedro\nand Yazidi, Anis\nand Zhang, Jianhua\nand Hammer, Hugo\nand Mello, Gustavo B. M.\nand Sandvig, Ioanna\nand Tufte, Gunnar\nand Nichele, Stefano',
       'Sidney Pontes{-}Filho and\nAnis Yazidi and\nJianhua Zhang and\nHugo Hammer and\nGustavo B. M. Mello and\nIoanna Sandvig and\nGunnar Tufte and\nStefano Nichele',
       'Aaby, Pernille\nand Biermann, Daniel\nand Yazidi, Anis\nand Mello, Gustavo Borges Moreno e.\nand Palumbo, Fabrizio',
       'Pontes-Filho, Sidney\nand Dahl, Annelene Gulden\nand Nichele, Stefano\nand Mello, Gustavo Borges Moreno e.',
       'Palumbo, Fabrizio and Buji, Adam and Yazidi, Anis and Haugerud, Hårek',
       'M. Fridman, R. K

In [15]:
def full_clean_authors(author_str):
    """
    Cleans and standardizes author names, resolving common formatting issues and known duplicates.

    Args:
        author_str: Raw author string from metadata.

    Returns:
        list: List of cleaned and normalized author names.
    """
    if pd.isna(author_str):
        return []

    # initial cleaning
    author_str = author_str.replace('\n', ' ').replace('{-}', '-')
    authors_raw = re.split(r'\s+and\s+', author_str)
    authors = [a.strip() for a in authors_raw if a.strip()]

    # normalize names
    normalized = []
    for name in authors:
        parts = re.split(r',| and ', name) if ',' in name and len(name.split()) > 4 else [name]
        for part in parts:
            part = part.strip()
            if ',' in part:
                last, first = part.split(',', 1)
                normalized.append(f"{first.strip()} {last.strip()}")
            else:
                normalized.append(part)

    # merge variants
    merged = []
    for name in normalized:
        name_clean = name.lower().strip()
        parts = name_clean.split()
        fuzzy_key = f"{parts[0][0]}_{parts[-1]}" if len(parts) > 1 else name_clean

        # manual fix for Gustavo due to problems
        if ("gustavo" in name_clean and "mello" in name_clean) or name_clean in {
            "mello", "g. mello", "g. b. m. mello", "gustavo b. m.",
            "gustavo borges mello", "gustavo moreno mello",
            "gustavo borges moreno e.", "gustavo borges moreno e. mello"
        }:
            canonical = "Gustavo Mello"

        # manual fix for Palumbo due to inconsistency
        elif ("palumbo" in name_clean and ("fabrizio" in name_clean or name_clean.startswith("f. palumbo"))) or name_clean in {
            "f. palumbo", "fabrizio palumbo"
        }:
            canonical = "Fabrizio Palumbo"

        else:
            existing = [a for a in normalized if a.lower().startswith(parts[0][0]) and a.lower().endswith(parts[-1])]
            canonical = max(existing, key=len) if existing else name

        merged.append(canonical)

    return merged


# apply full processing
df['cleaned_authors'] = df['authors'].apply(full_clean_authors)


In [16]:
for original, cleaned in zip(df['authors'], df['cleaned_authors']):
    print(f"\nOriginal: {original}\nCleaned: {cleaned}")


Original: Hagos, Desta Haileselassie and Løland, Martin and Yazidi, Anis and Kure, Øivind and Engelstad, Paal E.
Cleaned: ['Desta Haileselassie Hagos', 'Martin Løland', 'Anis Yazidi', 'Øivind Kure', 'Paal E. Engelstad']

Original: Riegler, M A and Stensen, M H and Witczak, O and Andersen, J M and Hicks, S A and Hammer, H L and Delbarre, E and Halvorsen, P and Yazidi, A and Holst, N and Haugen, T B
Cleaned: ['M A Riegler', 'M H Stensen', 'O Witczak', 'J M Andersen', 'S A Hicks', 'H L Hammer', 'E Delbarre', 'P Halvorsen', 'A Yazidi', 'N Holst', 'T B Haugen']

Original: Pontes-Filho, Sidney
and Lind, Pedro
and Yazidi, Anis
and Zhang, Jianhua
and Hammer, Hugo
and Mello, Gustavo B. M.
and Sandvig, Ioanna
and Tufte, Gunnar
and Nichele, Stefano
Cleaned: ['Sidney Pontes-Filho', 'Pedro Lind', 'Anis Yazidi', 'Jianhua Zhang', 'Hugo Hammer', 'Gustavo Mello', 'Ioanna Sandvig', 'Gunnar Tufte', 'Stefano Nichele']

Original: Sidney Pontes{-}Filho and
Anis Yazidi and
Jianhua Zhang and
Hugo Hammer and


## <a id="csv"></a> 3) Save DataFrame as CSV

[⬆️ Back to Top](#top)

In [17]:
# save cleaned df to csv file
df.to_csv('df_cleaned.csv', index=False)